In [8]:
# QDRANT
## Importar o client
from qdrant_client import QdrantClient
## Criar collection
from qdrant_client.http.models import Distance, VectorParams

# LANGCHAIN
## Importar Qdrant com Vector Store
from langchain.vectorstores import Qdrant
## Importar OpenAI embeddings
from langchain.embeddings.openai import OpenAIEmbeddings
## Função para criação de chunks
from langchain.text_splitter import CharacterTextSplitter
## Módulo para facilitar o uso de vector stores em QA (question answering)
from langchain.chains import RetrievalQA
## Importar LLM
from langchain.llms import OpenAI

# Variáveis de ambiente
import os
from dotenv import load_dotenv

In [4]:
client = QdrantClient(host='localhost', port=6333)

In [6]:
if not client.collection_exists('rag_qdrant_collection'):
    client.create_collection(
        collection_name='rag_qdrant_collection',
        vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
    )

In [11]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')

embeddings = OpenAIEmbeddings(
    api_key=openai_api_key,
    model='text-embedding-ada-002'
)

In [12]:
vectorstore = Qdrant(
    client=client,
    collection_name='rag_qdrant_collection',
    embeddings=embeddings
)

C:\Users\maldi\AppData\Local\Temp\ipykernel_22296\4162862089.py:1: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vectorstore = Qdrant(


In [13]:
def get_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [15]:
with open("./files/data.txt") as f:
    raw_text = f.read()

In [18]:
texts = get_chunks(raw_text)

vectorstore.add_texts(texts)

['7a001245e9a04dbc9781e3d76fa32e7b',
 'e1dd4b1702ce4728a80722b337a321f3',
 '0aeaeb1ba5804cd3aad2c09d819a6eb0',
 '7b55a0407c314554b5135776eba43e27',
 'caf02e0e2963439ba8f977b039ef35a3',
 '8c30848e7ff64fe4970cb85eda36d4c4',
 '2183eadd9a044c95a98591a525625ed3']

In [19]:
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

C:\Users\maldi\AppData\Local\Temp\ipykernel_22296\3755277632.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm=OpenAI(),


In [21]:
query = "O que é o Qdrant?"

response = qa.run(query)

print(response)

C:\Users\maldi\AppData\Local\Temp\ipykernel_22296\3167203859.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa.run(query)


 O Qdrant é um mecanismo de banco de dados vetorial open-source, projetado para armazenar, indexar e buscar vetores de alta dimensão.


In [22]:
query = "O que é um banco de dados vetorial?"

response = qa.run(query)

print(response)

 Um banco de dados vetorial é um tipo de banco de dados especializado em armazenar e recuperar vetores, que são representações numéricas de informações geradas por modelos de machine learning. Esses vetores costumam ter centenas ou milhares de dimensões e são usados para medir a similaridade entre dados.
